<a href="https://colab.research.google.com/github/HouriehSa/HouriehSa/blob/main/Torch_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
from torch import nn
import torch.nn.functional as F

In [ ]:
print(torch.__version__)

2.0.1+cu118


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
torch.cuda.get_device_name()

'Tesla T4'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class MnistClassifier(nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    self.flatten=nn.Flatten()
    self.model = nn.Sequential(
        nn.Linear(28*28, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size,  2*hidden_size),
        nn.ReLU(),
        nn.Linear(2*hidden_size, 10),
    )

  def forward(self, x):
    x=self.flatten(x)
    logits=self.model(x)
    return logits


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()])

mnist_train = datasets.MNIST(root="./data", download=True, train=True, transform=transform)
mnist_test = datasets.MNIST(root="./data", download=True, train=False, transform=transform)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(mnist_train, batch_size=16, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=16, shuffle=True)

In [ ]:
model = MnistClassifier(512)
model.to(device)
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


def train(dataloader, model, ce_loss, optimizer,epochs=100):
    size = len(dataloader.dataset)
    model.train()

    for epoch in range(epochs):

      print(f"{epoch}/100:")
      print("-" * 50)
      for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = ce_loss(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
train(train_loader, model, ce_loss, optimizer,epochs=10)

0/100:
--------------------------------------------------
loss: 2.297978  [   16/60000]
loss: 0.088379  [ 1616/60000]
loss: 0.121297  [ 3216/60000]
loss: 0.438661  [ 4816/60000]
loss: 0.562526  [ 6416/60000]
loss: 0.263411  [ 8016/60000]
loss: 0.060064  [ 9616/60000]
loss: 0.195325  [11216/60000]
loss: 0.397921  [12816/60000]
loss: 0.151716  [14416/60000]
loss: 0.075710  [16016/60000]
loss: 0.786295  [17616/60000]
loss: 0.035094  [19216/60000]
loss: 0.017111  [20816/60000]
loss: 0.178793  [22416/60000]
loss: 0.205581  [24016/60000]
loss: 0.429870  [25616/60000]
loss: 0.111984  [27216/60000]
loss: 0.003780  [28816/60000]
loss: 0.066809  [30416/60000]
loss: 0.420215  [32016/60000]
loss: 1.021928  [33616/60000]
loss: 0.012285  [35216/60000]
loss: 0.028876  [36816/60000]
loss: 0.090576  [38416/60000]
loss: 0.002948  [40016/60000]
loss: 0.041426  [41616/60000]
loss: 0.036864  [43216/60000]
loss: 0.028144  [44816/60000]
loss: 0.050642  [46416/60000]
loss: 0.057928  [48016/60000]
loss: 0.0285

In [ ]:

model.eval()

for x, y in test_loader:
  x, y = x.to(device), y.to(device)
  y_hat = model(x)
  print(f"The prediction: {torch.argmax(y_hat, dim=1)}")
  print(f"The ground truth: {y}")

The prediction: tensor([9, 9, 1, 9, 1, 0, 6, 6, 1, 1, 3, 6, 2, 6, 9, 0], device='cuda:0')
The ground truth: tensor([9, 3, 1, 9, 1, 0, 6, 6, 1, 1, 3, 6, 2, 6, 4, 0], device='cuda:0')
The prediction: tensor([7, 8, 0, 3, 0, 2, 6, 7, 3, 3, 2, 2, 6, 6, 1, 1], device='cuda:0')
The ground truth: tensor([7, 8, 0, 3, 0, 2, 6, 7, 3, 3, 2, 2, 6, 6, 1, 1], device='cuda:0')
The prediction: tensor([1, 9, 1, 0, 8, 3, 1, 9, 9, 5, 0, 0, 1, 7, 4, 9], device='cuda:0')
The ground truth: tensor([1, 9, 1, 0, 8, 3, 1, 9, 9, 5, 0, 0, 1, 7, 4, 9], device='cuda:0')
The prediction: tensor([9, 4, 0, 2, 7, 7, 5, 6, 3, 4, 7, 2, 6, 3, 1, 9], device='cuda:0')
The ground truth: tensor([9, 4, 0, 2, 7, 7, 5, 6, 3, 4, 7, 2, 6, 3, 1, 9], device='cuda:0')
The prediction: tensor([4, 2, 0, 3, 0, 7, 0, 6, 1, 4, 5, 5, 4, 0, 5, 0], device='cuda:0')
The ground truth: tensor([4, 2, 0, 3, 0, 7, 0, 6, 1, 4, 5, 5, 4, 0, 5, 0], device='cuda:0')
The prediction: tensor([5, 7, 8, 5, 4, 9, 8, 8, 9, 5, 1, 7, 4, 2, 4, 4], device='cuda:0')
